<a href="https://colab.research.google.com/github/satyamk795/multi-language-translaslator/blob/main/LanguageFlow_AI_Powered_Multi_Language_Translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LanguageFlow : AI-Powered Multi-Language Translator**

 LanguageFlow is a multilingual translation system that delivers human-quality translations across languages like English, Hindi, Spanish, French, and German. Leveraging advanced transformer models, this tool provides instant, context-aware translations while preserving cultural nuances and idiomatic expressions.The system features a lightweight architecture optimized for CPU deployment, ensuring fast performance without compromising accuracy. With its clean Gradio interface and optional privacy-focused local processing, LanguageFlow offers seamless cross-lingual interactions.

**STEP 1: INSTALL REQUIRED PACKAGES**

In [ ]:
!pip install transformers torch --quiet
print("✓ Packages installed successfully")

**STEP 2: SETUP TRANSLATION MODELS**

In [ ]:
from transformers import pipeline
import torch
import gradio as gr

DIRECT_TRANSLATIONS = {
    ('English', 'Hindi'): 'Helsinki-NLP/opus-mt-en-hi',
    ('Hindi', 'English'): 'Helsinki-NLP/opus-mt-hi-en',
    ('English', 'French'): 'Helsinki-NLP/opus-mt-en-fr',
    ('French', 'English'): 'Helsinki-NLP/opus-mt-fr-en',
    ('English', 'Spanish'): 'Helsinki-NLP/opus-mt-en-es',
    ('Spanish', 'English'): 'Helsinki-NLP/opus-mt-es-en',
    ('English', 'German'): 'Helsinki-NLP/opus-mt-en-de',
    ('German', 'English'): 'Helsinki-NLP/opus-mt-de-en',
    ('French', 'Spanish'): 'Helsinki-NLP/opus-mt-fr-es',
    ('Spanish', 'French'): 'Helsinki-NLP/opus-mt-es-fr',
}

SUPPORTED_LANGUAGES = ['English', 'Hindi', 'French', 'Spanish', 'German']

translators = {}
print("\n🔄 LOADING DIRECT TRANSLATION MODELS...")
for (src, tgt), model_name in DIRECT_TRANSLATIONS.items():
    try:
        print(f"• Loading {src} → {tgt}...", end=' ')
        translators[(src, tgt)] = pipeline(
            "translation",
            model=model_name,
            device=-1
        )
        print("✓ Success")
    except Exception as e:
        print(f"✗ Failed: {str(e)}")

**STEP 3: TRANSLATION FUNCTION**

In [ ]:
def translate(text, source_lang, target_lang):
    """Handles translation without path labels"""
    if not text.strip():
        return "Please enter text to translate"

    if (source_lang, target_lang) in translators:
        try:
            result = translators[(source_lang, target_lang)](text, max_length=400)
            return result[0]['translation_text']
        except Exception as e:
            return f"Translation error: {str(e)}"


    if (source_lang, 'English') in translators and ('English', target_lang) in translators:
        try:
            step1 = translators[(source_lang, 'English')](text, max_length=400)[0]['translation_text']
            step2 = translators[('English', target_lang)](step1, max_length=400)[0]['translation_text']
            return step2
        except Exception as e:
            return f"Translation error: {str(e)}"

    if (source_lang, 'French') in translators and ('French', target_lang) in translators:
        try:
            step1 = translators[(source_lang, 'French')](text, max_length=400)[0]['translation_text']
            step2 = translators[('French', target_lang)](step1, max_length=400)[0]['translation_text']
            return step2
        except Exception as e:
            return f"Translation error: {str(e)}"

    return f"Translation from {source_lang} to {target_lang} not available"

**STEP 4: GRADIO INTERFACE**

In [ ]:
with gr.Blocks(title="AI-Powered Multi-Language Translator") as app:
    gr.Markdown("# 🌍 AI-Powered Multi-Language Translator")

    with gr.Row():
        with gr.Column():
            source_lang = gr.Dropdown(
                label="From Language",
                choices=SUPPORTED_LANGUAGES,
                value="Select source language..."
            )
            input_text = gr.Textbox(
                label="Original Text",
                placeholder="Enter text here...",
                lines=5
            )

        with gr.Column():
            target_lang = gr.Dropdown(
                label="To Language",
                choices=SUPPORTED_LANGUAGES,
                value="Select target language..."
            )
            output_text = gr.Textbox(
                label="Translation Result",
                interactive=False,
                lines=5
            )

    translate_btn = gr.Button("Translate", variant="primary")

    translate_btn.click(
        fn=translate,
        inputs=[input_text, source_lang, target_lang],
        outputs=output_text
    )

**STEP 5: LAUNCHING THE APP**

In [ ]:
print("\n Starting translator app...")
app.launch(share=True)